## Parallel Execution of multiple Nodes
Parallel execution in LangChain refers to running multiple computation nodes (tasks) simultaneously instead of sequentially.

Each node can represent a distinct part of a workflow — such as calling different APIs, running multiple retrievals, or executing sub-models.
LangChain's **RunnableParallel** enables this kind of parallelism, allowing independent branches to execute concurrently, reducing overall latency.

⚙️ **Scenario: Real-Life Use Case**

A system receives a breaking news headline and must:

1. Summarize the event using an LLM.

2. Analyze its sentiment.

3. Extract key entities (people, organizations, places).

Instead of running these one by one, all are executed in parallel using RunnableParallel.
This demonstrates a practical **multi-agent workflow.**

In [19]:
from langchain_openai import ChatOpenAI
from langchain.schema.runnable import RunnableLambda, RunnableParallel
from langchain.prompts import ChatPromptTemplate
import time

In [2]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [ ]:
# Example headline
news_headline = """
Tesla shares surge after record-breaking quarterly profits and strong 
EV demand forecast.
"""

#### Prompts

In [4]:
# Prompt for summarization
summary_prompt = ChatPromptTemplate.from_template(
    "Summarize the following headline in one concise sentence:\n\n{headline}"
)

# Prompt for sentiment analysis
sentiment_prompt = ChatPromptTemplate.from_template(
    "Analyze the sentiment of this headline as Positive, Neutral, or Negative, and briefly explain why:\n\n{headline}"
)

# Prompt for entity extraction
entity_prompt = ChatPromptTemplate.from_template(
    "Extract key entities (like companies, people, and locations) from this headline. Return a JSON list.\n\n{headline}"
)


#### Runnable Pipelines (Nodes)

In [5]:
# Each node runs independently

# Summarization Node
summarizer = summary_prompt | llm

# Sentiment Analysis Node
sentiment_analyzer = sentiment_prompt | llm

# Entity Extraction Node
entity_extractor = entity_prompt | llm


#### Sequential Execution

In [20]:
# Run tasks one by one
start_time = time.perf_counter()

summary_res = summarizer.invoke({"headline": news_headline})
sentiment_res = sentiment_analyzer.invoke({"headline": news_headline})
entity_res = entity_extractor.invoke({"headline": news_headline})

sequential_result = {
    "summary": summary_res.content.strip(),
    "sentiment": sentiment_res.content.strip(),
    "entities": entity_res.content.strip(),
}

sequential_time = time.perf_counter() - start_time
print(f"Sequential Execution Time: {sequential_time:.2f} seconds")
sequential_result


Sequential Execution Time: 7.08 seconds


{'summary': 'Tesla shares rise sharply following record quarterly profits and a positive outlook for electric vehicle demand.',
 'sentiment': 'The sentiment of the headline is Positive. \n\nThis is indicated by the phrases "surge," "record-breaking quarterly profits," and "strong EV demand forecast," all of which convey a sense of success and optimism. The use of "surge" suggests a significant increase in share value, while "record-breaking" implies exceptional performance. Additionally, a "strong EV demand forecast" indicates positive future prospects for the company, contributing to an overall favorable sentiment.',
 'entities': '```json\n[\n    {\n        "entity": "Tesla",\n        "type": "company"\n    },\n    {\n        "entity": "quarterly profits",\n        "type": "financial term"\n    },\n    {\n        "entity": "EV demand",\n        "type": "market trend"\n    }\n]\n```'}

#### Combine All Nodes Using RunnableParallel

In [6]:
# Combine tasks into a parallel pipeline
parallel_tasks = RunnableParallel(
    summary=summarizer,
    sentiment=sentiment_analyzer,
    entities=entity_extractor
)

#### Execute All Tasks in Parallel

In [21]:
# Execute parallel tasks
start_time = time.perf_counter()
parallel_result = parallel_tasks.invoke({"headline": news_headline})
parallel_time = time.perf_counter() - start_time

# Clean up results
parallel_result_clean = {
    k: v.content.strip() for k, v in parallel_result.items()
}

print(f"Parallel Execution Time: {parallel_time:.2f} seconds")
parallel_result_clean

Parallel Execution Time: 2.96 seconds


{'summary': 'Tesla shares rise sharply following record quarterly profits and a positive outlook for electric vehicle demand.',
 'sentiment': 'The sentiment of the headline is Positive. \n\nThis is indicated by the use of words like "surge," "record-breaking," and "strong," which convey a sense of success and optimism. The mention of "quarterly profits" suggests financial growth, while "strong EV demand forecast" implies a positive outlook for the company\'s future performance. Overall, the language used in the headline reflects favorable news for Tesla, contributing to a positive sentiment.',
 'entities': '```json\n[\n    {\n        "entity": "Tesla",\n        "type": "company"\n    },\n    {\n        "entity": "quarterly profits",\n        "type": "financial term"\n    },\n    {\n        "entity": "EV demand",\n        "type": "market trend"\n    }\n]\n```'}

#### Async Execution (Improves Latency)

In [22]:
async def run_async_parallel():
    start = time.perf_counter()
    result = await parallel_tasks.ainvoke({"headline": news_headline})
    end = time.perf_counter()
    return {
        "results": {k: v.content.strip() for k, v in result.items()},
        "time": end - start,
    }

async_result = await run_async_parallel()

print(f"Async Parallel Execution Time: {async_result['time']:.2f} seconds")
async_result["results"]


Async Parallel Execution Time: 2.52 seconds


{'summary': 'Tesla shares rise sharply following record quarterly profits and a positive outlook for electric vehicle demand.',
 'sentiment': 'The sentiment of the headline is Positive. \n\nThis is because it highlights a significant achievement for Tesla, specifically "record-breaking quarterly profits," which indicates strong financial performance. Additionally, the mention of a "strong EV demand forecast" suggests optimism about future sales and growth in the electric vehicle market. Both elements contribute to a favorable outlook for the company, leading to a positive sentiment overall.',
 'entities': '```json\n[\n    {\n        "entity": "Tesla",\n        "type": "company"\n    },\n    {\n        "entity": "quarterly profits",\n        "type": "financial term"\n    },\n    {\n        "entity": "EV demand",\n        "type": "market trend"\n    }\n]\n```'}